In [ ]:
library(biomaRt)
library(tidyverse)
library(SignacX)

library(httr)
set_config( config( ssl_verifypeer = 0L ) )

In [ ]:
### TC_DEgenes :: cynologus to human
C_H_TCDEG_g1 = c('cyno','human','/DEmRNA/group1_padj_score.txt','/DEmRNA/biomaRt/human_group1_padj_score.txt')
C_H_TCDEG_g2 = c('cyno','human','/DEmRNA/group2_padj_score.txt','/DEmRNA/biomaRt/human_group2_padj_score.txt')
C_H_TCDEG_g3 = c('cyno','human','/DEmRNA/group3_padj_score.txt','/DEmRNA/biomaRt/human_group3_padj_score.txt')
C_H_TCDEG_g4 = c('cyno','human','/DEmRNA/group4_padj_score.txt','/DEmRNA/biomaRt/human_group4_padj_score.txt')
C_H_TCDEG_g5 = c('cyno','human','/DEmRNA/group5_padj_score.txt','/DEmRNA/biomaRt/human_group5_padj_score.txt')
C_H_TCDEG_g6 = c('cyno','human','/DEmRNA/group6_padj_score.txt','/DEmRNA/biomaRt/human_group6_padj_score.txt')
C_H_TCDEG_g7 = c('cyno','human','/DEmRNA/group7_padj_score.txt','/DEmRNA/biomaRt/human_group7_padj_score.txt')

### WGCNA_TC_DEgenes :: cynologus to human
C_H_WTCDEG_g2 = c('cyno','human','/DEmRNA/WGCNA_group2_padj_score.txt','/DEmRNA/biomaRt/human_WGCNA_group2_padj_score.txt')
C_H_WTCDEG_g3 = c('cyno','human','/DEmRNA/WGCNA_group3_padj_score.txt','/DEmRNA/biomaRt/human_WGCNA_group3_padj_score.txt')
C_H_WTCDEG_g6 = c('cyno','human','/DEmRNA/WGCNA_group6_padj_score.txt','/DEmRNA/biomaRt/human_WGCNA_group6_padj_score.txt')
C_H_WTCDEG_g7 = c('cyno','human','/DEmRNA/WGCNA_group7_padj_score.txt','/DEmRNA/biomaRt/human_WGCNA_group7_padj_score.txt')

### TIMER :: cynologus to human
C_H_TPM_sample = c('cyno','human','/batch_correction/batch_cor_GeneTPM_TIMER_sample_mfa.txt','/TIMER/human_batch_cor_GeneTPM_TIMER_sample_mfa.txt')
C_H_TPM_group  = c('cyno','human','/batch_correction/batch_cor_GeneTPM_TIMER_group_mfa.txt','/TIMER/human_batch_cor_GeneTPM_TIMER_group_mfa.txt')

### immun mark genes :: rhesus to cyno
R_C_immune = c('rhesus','cyno','/annotation_files/immune_mark_rhesus.txt','/annotation_files/immune_mark_cyno.txt')

### hubgenes :: cynologus to rhesus
C_R_hCC = c('cyno','rhesus','/hubgene/hubgene_cancer.txt','/hubgene/biomaRt/hubgene_cancer_rhesus.txt')
C_R_h20 = c('cyno','rhesus','/hubgene/hubgene_20.txt','/hubgene/biomaRt/hubgene_20_rhesus.txt')
C_R_h605 = c('cyno','rhesus','/hubgene/hubgene_605.txt','/hubgene/biomaRt/hubgene_605_rhesus.txt')

### hubgenes :: cynologus to human
C_H_hCC = c('cyno','human','/hubgene/hubgene_cancer.txt','/hubgene/biomaRt/hubgene_cancer_human.txt')
C_H_h20 = c('cyno','human','/hubgene/hubgene_20.txt','/hubgene/biomaRt/hubgene_20_human.txt')
C_H_h605 = c('cyno','human','/hubgene/hubgene_605.txt','/hubgene/biomaRt/hubgene_605_human.txt')

FILEs=list(C_H_TCDEG_g1,C_H_TCDEG_g2,C_H_TCDEG_g3,C_H_TCDEG_g4,C_H_TCDEG_g5,C_H_TCDEG_g6,C_H_TCDEG_g7,
           C_H_WTCDEG_g2,C_H_WTCDEG_g3,C_H_WTCDEG_g6,C_H_WTCDEG_g7,
           C_H_TPM_sample,C_H_TPM_group,
           R_C_immune,
           C_R_hCC,C_R_h20,C_R_h605,
           C_H_hCC,C_H_h20,C_H_h605
          )

In [ ]:
for (i in FILEs){
    data <- read.table(paste0('./source_data',i[3]),header=T,sep='\t')
    Genes <- data$gene
    
    ##### useMart
    A <- ifelse(i[1] == 'cyno', 'mfascicularis_gene_ensembl',
                ifelse(i[1] == 'rhesus','mmulatta_gene_ensembl','hsapiens_gene_ensembl'))
    Acol <- ifelse(i[1] == 'cyno' | i[1] == 'rhesus', 'external_gene_name','hgnc_symbol')
    
    B <- ifelse(i[2] == 'cyno', 'mfascicularis_gene_ensembl',
                ifelse(i[2] == 'rhesus','mmulatta_gene_ensembl','hsapiens_gene_ensembl'))
    Bcol <- ifelse(i[2] == 'cyno' | i[2] == 'rhesus', 'external_gene_name','hgnc_symbol')    
    
    Amart <- useMart(host = "https://aug2020.archive.ensembl.org", biomart = "ENSEMBL_MART_ENSEMBL", dataset = A)
    Bmart <- useMart(host = "https://aug2020.archive.ensembl.org", biomart = "ENSEMBL_MART_ENSEMBL", dataset = B)

    orthologs <- getLDS(attributes = c(Acol), filters = c(Acol), values = Genes, mart = Amart,
                        attributesL = c(Bcol), martL = Bmart)
    orthologs <- as_tibble(orthologs)
    colnames(orthologs) <- c("ASymbol", "BSymbol")

    ##### replace
    data$gene <- unlist(data$gene)
    for(j in 1:nrow(orthologs)) {Agene <- as.character(orthologs[j,1])
                                 Bgene <- as.character(orthologs[j,2])
                                 data$gene[data$gene ==  Agene] <- Bgene}
    ##### save
    write.table(data, file=paste0('./source_data',i[4]), sep='\t',col.names = T, row.names = F)
}